In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers, models


import data_processing as dp

2024-11-25 19:33:39.091683: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 19:33:39.092271: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-25 19:33:39.095540: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-25 19:33:39.105368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732525419.119566  280281 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732525419.12

In [2]:
# Load dataset
dataset_name = "output_dataset.csv"
df = pd.read_csv(dataset_name)

In [3]:
# Process dataset
df = dp.add_headers(df)
df = dp.trim_id(df)
df = dp.mask_data(df)
df = dp.impute_nans(df)
df

,Ones,Twos,Threes,Fours,Fives,Sixes,Total,Bonus,3 of a Kind,4 of a Kind,...,Total Mask,Bonus Mask,3 of a Kind Mask,4 of a Kind Mask,Full House Mask,Small Straight Mask,Large Straight Mask,Chance Mask,Yahtzee Mask,Score Mask
0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
1,-100.0,-100.0,-100.0,-100.0,-100.0,18.0,-100.0,-100.0,-100.0,-100.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2,-100.0,-100.0,-100.0,-100.0,-100.0,18.0,-100.0,-100.0,25.0,-100.0,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
3,-100.0,-100.0,-100.0,12.0,-100.0,18.0,-100.0,-100.0,25.0,-100.0,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
4,4.0,-100.0,-100.0,12.0,-100.0,18.0,-100.0,-100.0,25.0,-100.0,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,2.0,-100.0,9.0,16.0,20.0,18.0,-100.0,-100.0,27.0,29.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1095,2.0,-100.0,9.0,16.0,20.0,18.0,-100.0,-100.0,27.0,29.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1096,2.0,0.0,9.0,16.0,20.0,18.0,65.0,35.0,27.0,29.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1097,2.0,0.0,9.0,16.0,20.0,18.0,65.0,35.0,27.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
# Split dataset
X, y = dp.split_xy(df)
X_train, X_test, y_train, y_test = dp.split_train_test(X, y, 0.2)

# Normalize data
X_train = dp.scale_data(X_train, True)
X_test = dp.scale_data(X_test, False)

y

0       271
1       271
2       271
3       271
4       271
       ... 
1094    273
1095    273
1096    273
1097    273
1098    273
Name: Score, Length: 1099, dtype: int64

In [8]:
# Deep Neural Network
model = models.Sequential([
    layers.InputLayer(shape=(31,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

2024-11-25 19:31:20.875532: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [11]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 255.7374 - val_loss: 243.9282
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 239.1165 - val_loss: 200.6959
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 178.9700 - val_loss: 109.8386
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 104.5580 - val_loss: 88.9701
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 87.1115 - val_loss: 77.2534
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 70.0529 - val_loss: 67.8455
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 60.5388 - val_loss: 61.8199
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53.8008 - val_loss: 56.4856
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 48.8595 - val_loss: 51.3568
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 44.8970 - val_loss: 46.7370
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 42.4900 - val_loss: 45.3536
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms

In [ ]:
# Test loss
test_loss = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 44.8802 
Test Loss: 42.41960144042969


In [ ]:
import random

# Make predictions on the test data, print random
random_index = random.randint(0, len(X_test) - 1)
predictions = model.predict(X_test)
print("Data: ", list(X_test.iloc[random_index]))
print("Prediction: ", predictions[random_index][0]) 
print("Actual: ", y_test.iloc[random_index])

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Data:  [2.0, 8.0, 9.0, 12.0, 15.0, -100.0, -100.0, -100.0, 16.0, 14.0, 25.0, 30.0, 0.0, 21.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Prediction:  194.85019
Actual:  205


In [ ]:
h5_filename = "output_model.keras"
model.save(h5_filename)

failed to lookup keras version from the file,
    this is likely a weight only file
